In [7]:
import pygrib

grbs = pygrib.open('temp')

In [8]:
grb = grbs.select(name='Temperature')[-1]

In [69]:
grb.data()[0].data[180,0]

201.71119140625

In [68]:
grb.data(lat1=0,lat2=0,lon1=0,lon2=0)

(masked_array(data=[[201.71119140625]],
              mask=[[False]],
        fill_value=9999.0),
 array([[0.]]),
 array([[0.]]))

In [3]:
for grb in grbs:
    print(grb)

1:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 202311050600:ens mean
2:Temperature:K (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 202311050600:ens mean
3:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 202311050600:ens mean
4:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 202311050600:ens mean
5:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 202311050600:ens mean
6:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 0 hrs:from 202311050600:ens mean
7:Temperature:K (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 0 hrs:from 202311050600:ens mean
8:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 0 hrs:from 202311050600:ens mean
9:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 5000 

In [8]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from datetime import datetime
import time
import pygrib
import pandas as pd

ct_df = pd.read_csv("uscities.csv")[:10000]
lls = list(zip(ct_df['lat'], ct_df['lng']))

dt = datetime.now()
file_dt = "".join(list(map(str,[dt.year,dt.month,f"{dt.day:02d}"])))

client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket = client.list_objects(Bucket='noaa-gefs-pds', Prefix = "gefs."+file_dt+"/", Delimiter="/",)

while True:
    if bucket.get('CommonPrefixes') is not None:
        fldr = bucket.get('CommonPrefixes')[-1].get('Prefix')
        break
    else:
        time.sleep(300)

cycle = fldr.split('/')[-2]
base = f'{fldr[:-1]}/atmos/pgrb2ap5/gec00.t{cycle}z.pgrb2a.0p50.f'
files = []
for i in range(0,243,3):
    files.append(base+f"{i:03d}")
for i in range(246,390,6):
    files.append(base+f"{i}")


def process(file, cycle):
    grbs = pygrib.open(file)
    grb = grbs.select(name='Temperature')[-1]
    data = grb.data()[0]
    
    def f(lat, lon):
        return round(abs(data[int(round(-2*lat)+180),int(round(2*lon))]*1.8-459.67-65),2)
    
    ct_df['dd_'+cycle] = [f(lat, lon) for lat,lon in lls]
    
    grbs.close()

# while files:
#     idx = int(files[0].split(".")[-1][-3:])
#     print(files[0])
#     try:
#         if idx%24==0 and idx>0:
#             ct_df['dd'] = (ct_df['dd_00']+ct_df['dd_03']+ct_df['dd_06']+ct_df['dd_09']+ct_df['dd_12']+ct_df['dd_15']+ct_df['dd_18']+ct_df['dd_21'])/8
#             break

#         client.download_file('noaa-gefs-pds', files[0], 'temp')
#         process('temp', f"{idx%24:02d}")
        
#     except Exception as e:
#         print(e)
#         # time.sleep(60)
#         break
#     del files[0]


In [4]:
ct_df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,...,id,dd_00,dd_03,dd_06,dd_09,dd_12,dd_15,dd_18,dd_21,dd
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18972871,10768.2,...,1840034016,20.80,19.08,15.92,15.99,14.01,11.60,8.18,7.05,14.07875
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,12121244,3267.6,...,1840020491,1.54,1.26,2.62,4.71,3.27,0.64,1.34,2.19,2.19625
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8595181,4576.6,...,1840000494,8.92,5.94,1.70,0.75,1.23,4.76,10.16,14.07,5.94125
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,5711945,4945.7,...,1840015149,5.66,7.74,10.18,9.75,10.29,10.18,10.36,10.05,9.27625
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7935,-96.7667,5668165,1522.2,...,1840019440,2.24,5.04,11.98,15.51,12.81,10.36,8.92,6.81,9.20875
5,Houston,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,5650910,1394.6,...,1840020925,2.42,4.32,11.26,14.43,11.91,6.76,4.96,4.11,7.52125
6,Philadelphia,Philadelphia,PA,Pennsylvania,42101,Philadelphia,40.0077,-75.1339,5512873,4544.9,...,1840000673,21.34,17.10,11.06,7.53,13.11,11.78,8.54,7.77,12.27875
7,Atlanta,Atlanta,GA,Georgia,13121,Fulton,33.7628,-84.4220,5046555,1419.9,...,1840013660,9.46,0.72,7.12,10.47,5.07,0.08,2.96,4.89,5.09625
8,Washington,Washington,DC,District of Columbia,11001,District of Columbia,38.9047,-77.0163,4810669,4434.0,...,1840006060,19.00,13.14,6.74,3.21,6.09,6.38,6.92,6.87,8.54375
9,Boston,Boston,MA,Massachusetts,25025,Suffolk,42.3188,-71.0852,4208580,5505.8,...,1840000455,23.86,21.96,21.14,19.77,18.15,16.46,15.02,12.81,18.64625


In [3]:
result_df = ct_df.groupby(['state_name', 'city'])[['population', 'dd']].apply(lambda x: (x['population'] * x['dd']/x['population'].sum()).sum()).reset_index()

result_df.columns = ['state_name', 'city', 'dd']
result_df


,state_name,city,dd
0,California,Los Angeles,2.19625
1,District of Columbia,Washington,8.54375
2,Florida,Miami,9.27625
3,Georgia,Atlanta,5.09625
4,Illinois,Chicago,5.94125
5,Massachusetts,Boston,18.64625
6,New York,New York,14.07875
7,Pennsylvania,Philadelphia,12.27875
8,Texas,Dallas,9.20875
9,Texas,Houston,7.52125


In [19]:
result_df = ct_df.groupby('state_name').apply(lambda x: (x['population'] * x['dd']).sum()).reset_index()
result_df.columns = ['state_name', 'dd_state']


In [19]:
states_to_regions = {"Washington": "Pacific", "Oregon":"Pacific","California":"Pacific","Nevada":"Mountain","Wyoming":"Mountain","Arizona":"Mountain","Montana":"Mountain","Idaho":"Mountain","Utah":"Mountain","Colorado":"Mountain","New Mexico":"Mountain","North Dakota":"Midwest","South Dakota":"Midwest","Nebraska":"Midwest","Kansas":"Midwest","Minnesota":"Midwest","Iowa":"Midwest","Missouri":"Midwest","Wisconsin":"Midwest","Michigan":"Midwest","Illinois":"Midwest","Indiana":"Midwest","Ohio":"Midwest","New York":"Northeast","Pennsylvania":"Northeast","New Jersey":"Northeast","Vermont":"Northeast","New Hampshire":"Northeast","Maine":"Northeast","Massachusetts":"Northeast","Rhode Island":"Northeast","Connecticut":"Northeast","Texas":"South","Oklahoma":"South","Arkansas":"South","Louisiana":"South","North Carolina":"South","South Carolina":"South","Florida":"South","Georgia":"South","West Virginia":"South","Virginia":"South","Delaware":"South","Maryland":"South","Tennessee":"South","Mississippi":"South","Alabama":"South","Kentucky":"South","District of Columbia":"South","Puerto Rico":"lol","Hawaii":"lol","Alaska":"lol"}


In [31]:
ct_df["region"] =[states_to_regions[x] for x in ct_df["state_name"]]

ct_df.drop(ct_df[ct_df["region"]=="lol"].index, inplace=True)

In [33]:
ct_df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id,region
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18972871,10768.2,shape,False,True,America/New_York,1,11229 11226 11225 11224 11223 11221 11220 1138...,1840034016,Northeast
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,12121244,3267.6,shape,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90035 90034 9003...,1840020491,Pacific
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8595181,4576.6,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494,Midwest
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,5711945,4945.7,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149,South
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7935,-96.7667,5668165,1522.2,shape,False,True,America/Chicago,1,75287 75098 75234 75254 75251 75252 75253 7503...,1840019440,South
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,New Llano,New Llano,LA,Louisiana,22115,Vernon,31.0970,-93.2767,2472,401.0,shape,False,True,America/Chicago,3,71446 71461,1840017111,South
9996,Carlisle-Rockledge,Carlisle-Rockledge,AL,Alabama,1055,Etowah,34.1140,-86.1191,2472,60.3,shape,False,True,America/Chicago,4,35956 35954,1840037805,South
9997,Marmora,Marmora,NJ,New Jersey,34009,Cape May,39.2594,-74.6570,2471,484.3,shape,False,False,America/New_York,3,08223,1840081012,Northeast
9998,Holcomb,Holcomb,KS,Kansas,20055,Finney,37.9859,-100.9936,2471,739.1,shape,False,True,America/Chicago,3,67851,1840007523,Midwest


In [8]:
ct_df["lol"] = [11.80007812499997, 0.07992187500002501, 17.920078124999975, 7.999921874999984, 2.8000781249999704, 0.259921874999975, 14.140078125000002, 11.44007812500007, 12.520078124999998, 15.580078124999943]

In [9]:
lls = zip(ct_df['lat'], ct_df['lng'])
data = grb.data()[0]

def f(lat, lon):
    print(data[int(round(-2*lat)+180),int(round(2*lon))])
    return abs(data[int(round(-2*lat)+180),int(round(2*lon))]*1.8-459.67-65)

ct_df['dd_00'] = [f(lat, lon) for lat,lon in lls]

284.927734375
291.527734375
281.527734375
295.927734375
289.927734375
291.627734375
283.627734375
285.127734375
284.527734375
282.82773437500003


In [10]:
ct_df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id,dd_00
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18972871,10768.2,shape,False,True,America/New_York,1,11229 11226 11225 11224 11223 11221 11220 1138...,1840034016,11.800078
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,12121244,3267.6,shape,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90035 90034 9003...,1840020491,0.079922
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8595181,4576.6,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494,17.920078
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,5711945,4945.7,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149,7.999922
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7935,-96.7667,5668165,1522.2,shape,False,True,America/Chicago,1,75287 75098 75234 75254 75251 75252 75253 7503...,1840019440,2.800078
5,Houston,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,5650910,1394.6,shape,False,True,America/Chicago,1,77069 77068 77061 77060 77063 77062 77065 7706...,1840020925,0.259922
6,Philadelphia,Philadelphia,PA,Pennsylvania,42101,Philadelphia,40.0077,-75.1339,5512873,4544.9,shape,False,True,America/New_York,1,19154 19151 19150 19153 19152 19102 19103 1910...,1840000673,14.140078
7,Atlanta,Atlanta,GA,Georgia,13121,Fulton,33.7628,-84.4220,5046555,1419.9,shape,False,True,America/New_York,1,30334 30331 30332 30309 30308 30303 30354 3032...,1840013660,11.440078
8,Washington,Washington,DC,District of Columbia,11001,District of Columbia,38.9047,-77.0163,4810669,4434.0,shape,False,True,America/New_York,1,20250 20010 20011 20012 20015 20520 20064 2041...,1840006060,12.520078
9,Boston,Boston,MA,Massachusetts,25025,Suffolk,42.3188,-71.0852,4208580,5505.8,shape,False,True,America/New_York,1,02120 02121 02122 02124 02125 02126 02127 0212...,1840000455,15.580078


In [51]:
grb.data(lat1=34, lat2=34.5, lon1=241.5, lon2=242)

(array([[297. , 298.8],
        [292.7, 294.9]]),
 array([[34.5, 34.5],
        [34. , 34. ]]),
 array([[241.5, 242. ],
        [241.5, 242. ]]))

In [9]:
ct_df["state_name"].unique()

array(['New York', 'California', 'Illinois', 'Florida', 'Texas',
       'Pennsylvania', 'Georgia', 'District of Columbia', 'Massachusetts',
       'Arizona', 'Michigan', 'Washington', 'Minnesota', 'Colorado',
       'Maryland', 'Nevada', 'Missouri', 'Oregon', 'Puerto Rico', 'Ohio',
       'Indiana', 'North Carolina', 'Virginia', 'Wisconsin',
       'Rhode Island', 'Utah', 'Tennessee', 'Kentucky', 'Oklahoma',
       'Connecticut', 'Louisiana', 'Hawaii', 'Nebraska', 'New Mexico',
       'Alabama', 'South Carolina', 'Iowa', 'Kansas', 'Arkansas', 'Idaho',
       'Mississippi', 'New Jersey', 'Alaska', 'New Hampshire', 'Maine',
       'North Dakota', 'West Virginia', 'South Dakota', 'Delaware',
       'Montana', 'Vermont', 'Wyoming'], dtype=object)